In [29]:
import torch
import pandas as pd
from torch.utils.data import Dataset
from skimage import io
import os
from torch.utils.data import DataLoader
from torch import nn
import torch.functional as f
from skimage import color
import numpy as np
from torchvision.transforms import ToTensor

In [51]:
class IDCDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img_path = os.path.join(self.root_dir, self.annotations.iloc[index, 0])
        image = io.imread(img_path)
        y_label = torch.tensor(int(self.annotations.iloc[index, 1]))

        if self.transform:
            image = self.transform(image)

        return image, y_label

In [52]:
dataset = IDCDataset(csv_file='cancers.csv', root_dir='dset', transform=ToTensor())

In [53]:
trainset, testset = torch.utils.data.random_split(dataset, [1500, 500])

In [54]:
trainloader = DataLoader(dataset=trainset, batch_size=1, shuffle=True)
validloader = DataLoader(dataset=trainset, batch_size=1, shuffle=True)

In [55]:
for x, y in trainloader:
    print(x)
    print(y)
    break

tensor([[[[0.9216, 0.9451, 0.9373,  ..., 0.9529, 0.9294, 0.9137],
          [0.9490, 0.9529, 0.9569,  ..., 0.9137, 0.9176, 0.8863],
          [0.9490, 0.9294, 0.9451,  ..., 0.9216, 0.9176, 0.8314],
          ...,
          [0.9451, 0.8353, 0.8275,  ..., 0.8078, 0.9059, 0.6471],
          [0.9490, 0.8980, 0.6314,  ..., 0.7529, 0.7490, 0.5059],
          [0.9451, 0.9255, 0.8078,  ..., 0.5451, 0.7137, 0.6941]],

         [[0.8745, 0.9451, 0.9373,  ..., 0.9490, 0.8980, 0.8667],
          [0.9294, 0.9373, 0.9451,  ..., 0.8863, 0.8745, 0.8235],
          [0.9451, 0.9216, 0.9098,  ..., 0.8745, 0.8667, 0.6902],
          ...,
          [0.9176, 0.7333, 0.7373,  ..., 0.4431, 0.7216, 0.3216],
          [0.9490, 0.8275, 0.5176,  ..., 0.3765, 0.3647, 0.1804],
          [0.9373, 0.8863, 0.7020,  ..., 0.1882, 0.2941, 0.2902]],

         [[0.9020, 0.9529, 0.9451,  ..., 0.9490, 0.9294, 0.8980],
          [0.9490, 0.9294, 0.9451,  ..., 0.9216, 0.8941, 0.8784],
          [0.9294, 0.9490, 0.9373,  ..., 0

In [197]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.c1 = nn.Conv2d(in_channels=3, out_channels=10, kernel_size=(5,5), stride=5)
        self.r1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=(2,2), stride=2)

        self.c2 = nn.Conv2d(in_channels=10, out_channels=20, kernel_size=(2, 2), stride=1)
        self.r2 = nn.ReLU()
        self.maxpool2=nn.MaxPool2d(kernel_size=(3, 3), stride=1)
        self.lin1 = nn.Linear(2 * 2 * 20, 150)
        self.lin2 = nn.Linear(150, 2)
        self.relu = nn.ReLU()
        self.logsoft = nn.LogSoftmax(dim=1)

    def forward(self, x):
        x = self.c1(x)
        x = self.r1(x)
        x = self.maxpool1(x)
        x = self.c2(x)
        x = self.r2(x)
        x = self.maxpool2(x)
        x = x.flatten(1)
        x = self.lin1(x)
        x = self.lin2(x)
        x = self.relu(x)
        x = self.logsoft(x)
        return torch.exp(x)

In [198]:
model = Net()

In [199]:
for x, y in trainloader:
    yhat = model(x)
    print(y)
    print(yhat)
    actual =
    break

tensor([0])
tensor([[0.5104, 0.4896]], grad_fn=<ExpBackward0>)
